## Import packages

In [1]:
import pyomo
import pyomo.opt
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
n_time = 24
country = "Portugal"; # Possible options: Portugal, Slovenia, Greece, Denmark, this if apply only to Slovenia and Denmark, because both countries have biomass

In [3]:
def _auxDictionary(a):
    temp_dictionary = {}
    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]
    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]
    return temp_dictionary
#temp_dict1 = _auxDictionary(loadLimit)

## Data

In [4]:
data = {}
data['Inputs'] = pd.read_csv('Inputs.csv')
data['EVs_Inputs'] = pd.read_csv('EVs_Inputs.csv')
data['alpha'] = pd.read_csv('alpha.csv')
data['PchmaxEV'] = pd.read_csv('PchmaxEV.csv')
data['S'] = pd.read_csv('S.csv')

In [5]:
n_evs = data['EVs_Inputs']['Esoc'].size

## Start Time

In [6]:
from datetime import datetime

now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

Start Time = 08:14:35


## Sets

In [7]:
model = pyo.ConcreteModel()

model.ev = pyo.Set(initialize = np.arange(1, n_evs + 1))
model.t = pyo.Set(initialize = np.arange(1, n_time + 1))


## Parameters

In [8]:
model.EEVmax = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,2]))
model.EEVmin = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,1]))
model.ESoc = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,0]))
model.dT = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,0]))
model.cDA = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,1]))
model.Pl = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.Php = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,3]))
model.Pb = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,4]))
model.Pimp = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,5]))
model.Ppv = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,6]))
model.Pwind = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,7]))
model.Phid = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,8]))
model.Pbio = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,9]))
model.PchmaxEV = pyo.Param(model.ev, model.t, initialize =_auxDictionary(data['PchmaxEV'].to_numpy()))
model.S = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['S'].to_numpy()))
model.alpha = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['alpha'].to_numpy()))
model.RealHour = pyo.Param(model.t, initialize=_auxDictionary(data['Inputs'].to_numpy()[:,9]))
model.penalty1 = 1000
model.penalty2 = 1000
model.penalty3 = 0.6
model.Etrip = pyo.Param(model.ev, initialize=_auxDictionary(data['EVs_Inputs'].to_numpy()[:,3]))
model.n = 0.95
model.m = 0.6
model.factor = 1

## Variables

In [9]:
model.PEV = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.PEVdc = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.EEV = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Etriprelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Eminsocrelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Etripn = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Pgen = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Pres = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Pex = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)

## Constraints

In [10]:
def _balance_etripn(m,ev,t): 
    return m.Etripn[ev,t] == m.Etrip[ev]*m.S[ev,t]/(sum([m.S[ev,k] for k in np.arange(1, n_time + 1)]))
model.balance_etripn = pyo.Constraint(model.ev, model.t, rule = _balance_etripn)

def _balance_energy_EVS3(m,ev,t): 
    if t == 24:
        return m.EEV[ev,t] + m.Etriprelax[ev,t] >= m.EEVmax[ev]*m.m
    return pyo.Constraint.Skip
model.balance_energy_EVS3 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS3)

def _balance_energy_EVS(m,ev,t): 
    if t == 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.ESoc[ev] + m.PEV[ev,t]*m.dT[t]*m.n - m.PEVdc[ev,t]*m.dT[t]/m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS)

def _balance_energy_EVS2(m,ev,t): 
    if t > 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.EEV[ev,t-1] + m.PEV[ev,t]*m.dT[t]*m.n - m.PEVdc[ev,t]*m.dT[t]/m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS2 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS2)

def _RES_balance(m,t):
    return m.Pres[t] == m.Pwind[t] + m.Ppv[t] + m.Phid[t] + m.Pbio[t]
model.RES_balance = pyo.Constraint(model.t, rule = _RES_balance)

def _total_balance(m,t):
    if (country == "Slovenia"):
        return m.Pgen[t] + m.Pres[t] + m.Pimp[t] + sum(m.PEVdc[ev,t] for ev in np.arange(1, n_evs + 1)) - m.Pex[t] == m.Pl[t] + m.Pb[t]  + sum(m.PEV[ev, t] for ev in np.arange(1, n_evs + 1))
    else:
        return m.Pgen[t] + m.Pres[t] + m.Pimp[t] + sum(m.PEVdc[ev,t] for ev in np.arange(1, n_evs + 1)) - m.Pex[t] == m.Pl[t] + m.Pb[t] + sum(m.PEV[ev, t] for ev in np.arange(1, n_evs + 1))
model.total_balance = pyo.Constraint(model.t, rule = _total_balance)

def _power_charging_limit1(m,ev,t): 
    return m.PEV[ev,t] >= 0
model.power_charging_limit1 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit1)

def _power_charging_limit2(m,ev,t): 
    return m.PEV[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_charging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit2)

def _power_discharging_limit2(m,ev,t): 
    return m.PEVdc[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_discharging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_discharging_limit2)

def _energy_limits_EVS_1(m,ev,t): 
    return m.EEVmin[ev] <= m.EEV[ev,t] + m.Eminsocrelax[ev,t]
    #return m.EEVmin[ev] <= m.EEV[ev,t]
model.energy_limits_EVS_1 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_1)

def _energy_limits_EVS_2(m,ev,t): 
    return m.EEV[ev,t] <= m.EEVmax[ev]
model.energy_limits_EVS_2 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_2)


## Objective function

In [11]:
def _FOag(m):
    return sum(m.Pgen[t] for t in np.arange(1, n_time + 1)) + sum([m.Etriprelax[ev,t]*m.penalty1 + m.Eminsocrelax[ev,t]*m.penalty2 for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

model.FOag = pyo.Objective(rule = _FOag, sense = pyo.minimize)

## Solve the model

In [12]:
from pyomo.opt import SolverFactory

model.write('res_V4_EC.lp',  io_options={'symbolic_solver_labels': True})

# Create a solver
#opt = pyo.SolverFactory('cbc', executable='C:/Program Files/Cbc-releases.2.10.8-x86_64-w64-mingw64/bin/cbc.exe')

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio129/cplex/bin/x64_win64/cplex.exe')
opt.options['LogFile'] = 'res_V4_EC.log'

#opt = pyo.SolverFactory('ipopt', executable='C:/Program Files/Ipopt-3.11.1-win64-intel13.1/bin/ipopt.exe')
#opt.options['print_level'] = 12
#opt.options['output_file'] = "res_V5_EC.log"

results = opt.solve(model)#, tee=True)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmponftd9ch
  Lower bound: 9945604.568676103
  Upper bound: 9945604.568676103
  Number of objectives: 1
  Number of constraints: 507049
  Number of variables: 432073
  Number of nonzeros: 1083097
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 7.45
  Termination condition: optimal
  Termination message: Dual simplex - Optimal\x3a Objective = 9.9456045687e+06
  Error rc: 0
  Time: 11.57652497291565
# ----------------------------------------------------------
#   Solution Information
# --------------

## Objective Function Value

In [13]:
pyo.value(model.FOag)

9945604.568676125

## End Time

In [14]:
now = datetime.now()

end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)
print("Dif: {}".format(datetime.strptime(end_time, "%H:%M:%S") - datetime.strptime(start_time, "%H:%M:%S")))

End Time = 08:16:21
Dif: 0:01:46


## Cost Value

In [15]:
def ext_pyomo_vals(vals):
    # make a pd.Series from each
    s = pd.Series(vals.extract_values(),
                  index=vals.extract_values().keys())
    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:    # it is multi-indexed
        s = s.unstack(level=1)
    else:
        # force transition from Series -> df
        s = pd.DataFrame(s)
    return s

In [16]:
PEV_df = ext_pyomo_vals(model.PEV)
dT_df = ext_pyomo_vals(model.dT)
cDA_df = ext_pyomo_vals(model.cDA)
PEVdc_df = ext_pyomo_vals(model.PEVdc)
EEV_df = ext_pyomo_vals(model.EEV)

charge_cost = sum([PEV_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                   for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

discharge_cost = sum([PEVdc_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                      for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

print('Charge cost: {}'.format(charge_cost))
print('Discharge cost: {}'.format(discharge_cost))

Charge cost: 1132013.0692250833
Discharge cost: 459552.476911195


## Results

In [17]:
print("Total Charge: {}".format(np.sum(PEV_df.to_numpy())/1000000))

Total Charge: 6.502977491703602


In [18]:
print("Total Discharge: {}".format(np.sum(PEVdc_df.to_numpy())/1000000))

Total Discharge: 2.1137093847625006


In [19]:
EEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,44521.23750,44521.237500,44521.237500,44521.237500,44521.23750,44521.2375,44521.2375,44521.237500,44521.237500,45957.525000,...,45478.762500,45478.762500,45478.762500,45478.762500,45478.762500,45478.762500,45478.762500,45478.762500,45478.762500,45000.0
2,9960.00000,9720.038824,9720.038824,9720.038824,10198.80000,10198.8000,10198.8000,10198.800000,10198.800000,10198.800000,...,10079.400000,10079.400000,10079.400000,10079.400000,10079.400000,10079.400000,10079.400000,10079.400000,10079.400000,9960.0
3,40200.00000,40200.000000,40200.000000,40200.000000,40200.00000,40200.0000,40200.0000,41630.553152,41630.553152,44067.846514,...,43418.513181,43418.513181,43418.513181,43418.513181,43418.513181,43418.513181,43418.513181,43418.513181,43418.513181,40200.0
4,45000.00000,45000.000000,45000.000000,45000.000000,45000.00000,45000.0000,45000.0000,45000.000000,45000.000000,45000.000000,...,45393.300000,45393.300000,45393.300000,45393.300000,45393.300000,45393.300000,45393.300000,45393.300000,45393.300000,45000.0
5,43596.39116,43596.391160,43596.391160,43596.391160,43596.39116,46295.4000,46295.4000,46295.400000,46295.400000,46295.400000,...,46247.700000,46247.700000,46247.700000,46247.700000,46247.700000,46247.700000,46247.700000,46247.700000,46247.700000,46200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,45000.00000,45000.000000,45000.000000,45000.000000,45000.00000,45000.0000,45000.0000,45000.000000,45000.000000,45000.000000,...,45508.500000,45508.500000,45508.500000,45508.500000,45508.500000,45508.500000,45508.500000,45508.500000,45508.500000,45000.0
2997,44749.25000,44749.250000,44749.250000,44749.250000,44749.25000,44749.2500,44749.2500,44749.250000,44749.250000,44749.250000,...,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.0
2998,42000.00000,42000.000000,42000.000000,42000.000000,42000.00000,42000.0000,45457.2000,45457.200000,45457.200000,45457.200000,...,44469.428571,43975.542857,43975.542857,43481.657143,43481.657143,43481.657143,42987.771429,42493.885714,42000.000000,42000.0
2999,44792.55000,44792.550000,44792.550000,44792.550000,44792.55000,44792.5500,44792.5500,44792.550000,44792.550000,44792.550000,...,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.0


In [20]:
PEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,2015.842105,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,503.959132,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1505.845423,0.0,3249.080732,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.000000,2841.061937,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2997,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,527.894737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2998,0.0,0.0,0.0,0.0,0.000000,0.000000,3639.157895,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2999,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,436.736842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
PEV_df.sum()

1     9.257968e+04
2     0.000000e+00
3     1.758523e+05
4     1.390388e+05
5     9.151462e+05
6     7.598667e+05
7     5.969473e+05
8     6.255258e+05
9     6.528364e+05
10    6.573320e+05
11    2.523918e+05
12    1.889220e+04
13    6.604366e+03
14    1.420385e+06
15    1.392754e+05
16    0.000000e+00
17    0.000000e+00
18    0.000000e+00
19    0.000000e+00
20    0.000000e+00
21    0.000000e+00
22    0.000000e+00
23    2.849148e+03
24    4.745489e+04
dtype: float64

In [22]:
PEVdc_df.sum()

1     263958.041398
2     232229.037579
3          0.000000
4          0.000000
5          0.000000
6          0.000000
7          0.000000
8          0.000000
9          0.000000
10         0.000000
11         0.000000
12         0.000000
13    383754.652500
14     62601.197500
15     45761.285000
16         0.000000
17         0.000000
18         0.000000
19         0.000000
20         0.000000
21         0.000000
22         0.000000
23    177762.722000
24    947642.448786
dtype: float64

In [23]:
Pgen_df = ext_pyomo_vals(model.Pgen)

Pgen_df

,0
1,2.553275e+05
2,0.000000e+00
3,0.000000e+00
4,0.000000e+00
5,0.000000e+00
6,0.000000e+00
7,0.000000e+00
8,0.000000e+00
9,0.000000e+00
10,0.000000e+00


In [24]:
Pres_df = ext_pyomo_vals(model.Pres)

Pres_df

,0
1,6.085227e+06
2,5.948526e+06
3,6.151688e+06
4,5.907329e+06
5,6.545075e+06
6,6.252979e+06
7,5.863696e+06
8,5.822859e+06
9,5.811374e+06
10,5.838394e+06


In [25]:
Etriprelax_df = ext_pyomo_vals(model.Etriprelax)

In [26]:
Etriprelax_df.sum()

1        0.00
2        0.00
3        0.00
4        0.00
5        0.00
6        0.00
7        0.00
8        0.00
9        0.00
10       0.00
11       0.00
12       0.00
13       0.00
14       0.00
15       0.00
16       0.00
17       0.00
18       0.00
19       0.00
20       0.00
21       0.00
22       0.00
23       0.00
24    1699.44
dtype: float64

In [27]:
Eminsocrelax_df = ext_pyomo_vals(model.Eminsocrelax)

In [28]:
Eminsocrelax_df.sum()

1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
dtype: float64

In [29]:
Etripn_df = ext_pyomo_vals(model.Etripn)

In [30]:
Etripn_df.sum()

1     306577.030530
2       5304.887131
3          0.000000
4          0.000000
5          0.000000
6          0.000000
7          0.000000
8          0.000000
9          0.000000
10    100109.908333
11    241018.800000
12         0.000000
13    152273.988853
14    599476.835983
15    259866.379538
16    317942.365149
17    103607.316435
18    201483.015792
19     66832.958288
20    210267.788436
21    254300.247633
22    219829.993476
23    257323.066999
24    660055.667423
dtype: float64

In [31]:
Pex_df = ext_pyomo_vals(model.Pex)

In [32]:
Pex_df

,0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0
10,0.0


## Save results in csv files

In [33]:
import os 
folder = 'results_RES_V2G_' + str(n_evs)

if not os.path.exists(folder):
    os.makedirs(folder)
    
EEV_df.to_csv(folder + '/EEV.csv')
PEV_df.to_csv(folder + '/PEV.csv')
PEVdc_df.to_csv(folder + '/PEVdc.csv')
PEV_df.sum().to_csv(folder + '/PEV_h.csv')
PEVdc_df.sum().to_csv(folder + '/PEVdc_h.csv')

Etriprelax_df.to_csv(folder + '/Etriprelax.csv')
Etriprelax_df.sum().to_csv(folder + '/Etriprelax_h.csv')

Eminsocrelax_df.to_csv(folder + '/Eminsocrelax.csv')
Eminsocrelax_df.sum().to_csv(folder + '/Eminsocrelax_h.csv')

Etripn_df.to_csv(folder + '/Etripn.csv')
Etripn_df.sum().to_csv(folder + '/Etripn_h.csv')

Pgen_df.to_csv(folder + '/Pgen.csv')
Pres_df.to_csv(folder + '/Pres.csv')
Pex_df.to_csv(folder + '/Pex.csv')